# What is this?
A notebook to experiment with the BVG (Berlin public transport) API.

https://v6.bvg.transport.rest/
rate limit: 100 req/min

In [6]:
# Find Kuno Fischer and Amtsgericht station ID (the latter only for direction)
#!pip install requests
import requests

# API URL
url = "https://v6.bvg.transport.rest/stops"

# Make the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
else:
    print(f"Error: {response.status_code}")

for d in data:
    if "Kuno-Fischer" in d["name"]:
        print(f"ID for '{d['name']}': '{d['id']}'")
    if "Amtsgerichtsplatz" in d["name"]:
        print(f"ID for '{d['name']}': '{d['id']}'")

ID for 'Kuno-Fischer-Str.': 'de:11000:900024151::1'
ID for 'Amtsgerichtsplatz': 'de:11000:900024104::1'
ID for 'Kuno-Fischer-Str.': 'de:11000:900024151::2'
ID for 'Amtsgerichtsplatz': 'de:11000:900024104::2'
ID for 'Amtsgerichtsplatz': 'de:11000:900024104::3'


In [2]:
# API endpoint
url = "https://v6.bvg.transport.rest/stops/900024151/departures"

# Query parameters (customize as needed)
params = {
    "direction": "900024104",  # Optional: Filter departures by a specific direction
    "duration": 30,  # Show departures for the next 10 minutes
    "remarks": True,  # Include warnings and hints
    "language": "en",  # Language of the results
    "pretty": True,  # Pretty-print JSON responses
}

# Send GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
    print(data)  # Print or process the data as needed
else:
    print(f"Error: {response.status_code} - {response.text}")

{'departures': [{'tripId': '1|57481|0|86|19032025', 'stop': {'type': 'stop', 'id': '900024151', 'name': 'Kuno-Fischer-Str. (Berlin)', 'location': {'type': 'location', 'id': '900024151', 'latitude': 52.506613, 'longitude': 13.289778}, 'products': {'suburban': False, 'subway': False, 'tram': False, 'bus': True, 'ferry': False, 'express': False, 'regional': False}}, 'when': None, 'plannedWhen': '2025-03-19T20:15:00+01:00', 'prognosedWhen': None, 'delay': None, 'platform': None, 'plannedPlatform': None, 'prognosedPlatform': None, 'prognosisType': None, 'direction': 'S+U Zoologischer Garten', 'provenance': None, 'line': {'type': 'line', 'id': 'de-vbb-11000000-bus-m49', 'fahrtNr': '148952', 'name': 'M49', 'public': True, 'adminCode': 'BVB---', 'productName': 'Bus', 'mode': 'bus', 'product': 'bus', 'operator': {'type': 'operator', 'id': 'berliner-verkehrsbetriebe', 'name': 'Berliner Verkehrsbetriebe'}}, 'remarks': [{'id': '261268', 'type': 'warning', 'summary': 'The trip is cancelled', 'text'

In [3]:
data["departures"][0]

{'tripId': '1|57481|0|86|19032025',
 'stop': {'type': 'stop',
  'id': '900024151',
  'name': 'Kuno-Fischer-Str. (Berlin)',
  'location': {'type': 'location',
   'id': '900024151',
   'latitude': 52.506613,
   'longitude': 13.289778},
  'products': {'suburban': False,
   'subway': False,
   'tram': False,
   'bus': True,
   'ferry': False,
   'express': False,
   'regional': False}},
 'when': None,
 'plannedWhen': '2025-03-19T20:15:00+01:00',
 'prognosedWhen': None,
 'delay': None,
 'platform': None,
 'plannedPlatform': None,
 'prognosedPlatform': None,
 'prognosisType': None,
 'direction': 'S+U Zoologischer Garten',
 'provenance': None,
 'line': {'type': 'line',
  'id': 'de-vbb-11000000-bus-m49',
  'fahrtNr': '148952',
  'name': 'M49',
  'public': True,
  'adminCode': 'BVB---',
  'productName': 'Bus',
  'mode': 'bus',
  'product': 'bus',
  'operator': {'type': 'operator',
   'id': 'berliner-verkehrsbetriebe',
   'name': 'Berliner Verkehrsbetriebe'}},
 'remarks': [{'id': '261268',
   't

In [ ]:
import pandas as pd
import datetime


def get_stop_data():
    # API endpoint
    url = "https://v6.bvg.transport.rest/stops/900024151/departures"

    # Query parameters (customize as needed)
    params = {
        "direction": "900024104",  # Optional: Filter departures by a specific direction
        "duration": 30,  # Show departures for the next 10 minutes
        "remarks": True,  # Include warnings and hints
        "language": "en",  # Language of the results
        "pretty": True,  # Pretty-print JSON responses
    }

    # Send GET request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
    else:
        print(f"Error: {response.status_code} - {response.text}")

    result = {
        "type": [],
        "line": [],
        "departure": [],
        "delay": [],
        "direction": [],
        "cancelled": [],
    }

    for bus in data["departures"]:
        assert bus["stop"]["products"]["bus"], "The heck? There can only be busses."

        result["type"].append("Bus")
        result["line"].append(bus["line"]["name"])
        result["departure"].append(
            datetime.datetime.fromisoformat(bus["plannedWhen"]).strftime("%H:%M:%S")
        )
        delay = 0 if bus["delay"] is None else bus["delay"]
        delay /= 60
        result["delay"].append(delay)
        result["direction"].append(bus["direction"])
        result["cancelled"].append(bus["cancelled"])

    updated_at = datetime.datetime.fromtimestamp(
        data["realtimeDataUpdatedAt"]
    ).strftime("%H:%M:%S")

    return updated_at, pd.DataFrame(result)

In [8]:
t, r = get_stop_data()
print(t)
print(r)

20:17:39
  type line departure  delay                direction  cancelled
0  Bus  M49  20:27:00    0.0  S+U Zoologischer Garten       True
1  Bus  M49  20:37:00    0.0  S+U Zoologischer Garten       True
2  Bus  M49  20:47:00    0.0  S+U Zoologischer Garten       True
